In [ ]:
import re
import time
import random
from typing import Dict, List, Tuple, Literal
from dataclasses import dataclass, field

@dataclass
class QueryAnalysis:
    query: str
    complexity: Literal["simple", "medium", "complex"]
    strategy: Literal["fast", "cot", "tool"]
    confidence: float
    reasoning: str
    execution_time: float = 0.0
    success: bool = True

class MetaReasoningController:
    def __init__(self):
        self.query_history: List[QueryAnalysis] = []
        self.patterns = {
            'math': r'(\d+\s*[\+\-\*\/]\s*\d+)|calculate|compute|sum|product',
            'search': r'current|latest|news|today|who is|what is.*now',
            'creative': r'write|poem|story|joke|imagine',
            'logical': r'if.*then|because|therefore|prove|explain why',
            'simple_fact': r'^(what|who|when|where) (is|are|was|were)',
        }

    def analyze_query(self, query: str) -> QueryAnalysis:
        query_lower = query.lower()
        has_math = bool(re.search(self.patterns['math'], query_lower))
        needs_search = bool(re.search(self.patterns['search'], query_lower))
        is_creative = bool(re.search(self.patterns['creative'], query_lower))
        is_logical = bool(re.search(self.patterns['logical'], query_lower))
        is_simple = bool(re.search(self.patterns['simple_fact'], query_lower))
        word_count = len(query.split())
        has_multiple_parts = '?' in query[:-1] or ';' in query

        if has_math:
            complexity = "medium"
            strategy = "tool"
            reasoning = "Math detected - using calculator tool for accuracy"
            confidence = 0.9
        elif needs_search:
            complexity = "medium"
            strategy = "tool"
            reasoning = "Current/dynamic info - needs search tool"
            confidence = 0.85
        elif is_simple and word_count < 10:
            complexity = "simple"
            strategy = "fast"
            reasoning = "Simple factual query - fast retrieval sufficient"
            confidence = 0.95
        elif is_logical or has_multiple_parts or word_count > 30:
            complexity = "complex"
            strategy = "cot"
            reasoning = "Complex reasoning required - using chain-of-thought"
            confidence = 0.8
        elif is_creative:
            complexity = "medium"
            strategy = "cot"
            reasoning = "Creative task - chain-of-thought for idea generation"
            confidence = 0.75
        else:
            complexity = "medium"
            strategy = "cot"
            reasoning = "Unclear complexity - defaulting to chain-of-thought"
            confidence = 0.6

        return QueryAnalysis(query, complexity, strategy, confidence, reasoning)

In [ ]:
class FastHeuristicEngine:
    def __init__(self):
        self.knowledge_base = {
            'capital of france': 'Paris',
            'capital of spain': 'Madrid',
            'speed of light': '299,792,458 meters per second',
            'boiling point of water': '100°C or 212°F at sea level',
        }
    def answer(self, query: str) -> str:
        q = query.lower()
        for k, v in self.knowledge_base.items():
            if k in q:
                return f"Answer: {v}"
        if 'hello' in q or 'hi' in q:
            return "Hello! How can I help you?"
        return "Fast heuristic: No direct match found."

class ChainOfThoughtEngine:
    def answer(self, query: str) -> str:
        s = []
        s.append("Step 1: Understanding the question")
        s.append(f"  → The query asks about: {query[:50]}...")
        s.append("\nStep 2: Breaking down the problem")
        if 'why' in query.lower():
            s.append("  → This is a causal question requiring explanation")
            s.append("  → Need to identify causes and effects")
        elif 'how' in query.lower():
            s.append("  → This is a procedural question")
            s.append("  → Need to outline steps or mechanisms")
        else:
            s.append("  → Analyzing key concepts and relationships")
        s.append("\nStep 3: Synthesizing answer")
        s.append("  → Combining insights from reasoning steps")
        s.append("\nStep 4: Final answer")
        s.append("  → [Detailed response based on reasoning chain]")
        return "\n".join(s)

class ToolExecutor:
    def calculate(self, expression: str) -> float:
        m = re.search(r'(\d+\.?\d*)\s*([\+\-\*\/])\s*(\d+\.?\d*)', expression)
        if m:
            a, op, b = m.groups()
            a, b = float(a), float(b)
            ops = {
                '+': lambda x, y: x + y,
                '-': lambda x, y: x - y,
                '*': lambda x, y: x * y,
                '/': lambda x, y: x / y if y != 0 else float('inf'),
            }
            return ops[op](a, b)
        return None

    def search(self, query: str) -> str:
        return f"[Simulated search results for: {query}]"

    def execute(self, query: str, tool_type: str) -> str:
        if tool_type == "calculator":
            r = self.calculate(query)
            if r is not None:
                return f"Calculator result: {r}"
            return "Could not parse mathematical expression"
        elif tool_type == "search":
            return self.search(query)
        return "Tool execution completed"

In [ ]:
class MetaReasoningAgent:
    def __init__(self):
        self.controller = MetaReasoningController()
        self.fast_engine = FastHeuristicEngine()
        self.cot_engine = ChainOfThoughtEngine()
        self.tool_executor = ToolExecutor()
        self.stats = {
            'fast': {'count': 0, 'total_time': 0},
            'cot': {'count': 0, 'total_time': 0},
            'tool': {'count': 0, 'total_time': 0},
        }

    def process_query(self, query: str, verbose: bool = True) -> str:
        if verbose:
            print("\n" + "="*60)
            print(f"QUERY: {query}")
            print("="*60)
        t0 = time.time()
        analysis = self.controller.analyze_query(query)

        if verbose:
            print(f"\n🧠 META-REASONING:")
            print(f"   Complexity: {analysis.complexity}")
            print(f"   Strategy: {analysis.strategy.upper()}")
            print(f"   Confidence: {analysis.confidence:.2%}")
            print(f"   Reasoning: {analysis.reasoning}")
            print(f"\n⚡ EXECUTING {analysis.strategy.upper()} STRATEGY...\n")

        if analysis.strategy == "fast":
            resp = self.fast_engine.answer(query)
        elif analysis.strategy == "cot":
            resp = self.cot_engine.answer(query)
        elif analysis.strategy == "tool":
            if re.search(self.controller.patterns['math'], query.lower()):
                resp = self.tool_executor.execute(query, "calculator")
            else:
                resp = self.tool_executor.execute(query, "search")

        dt = time.time() - t0
        analysis.execution_time = dt
        self.stats[analysis.strategy]['count'] += 1
        self.stats[analysis.strategy]['total_time'] += dt
        self.controller.query_history.append(analysis)

        if verbose:
            print(resp)
            print(f"\n⏱️  Execution time: {dt:.4f}s")
        return resp

    def show_stats(self):
        print("\n" + "="*60)
        print("AGENT PERFORMANCE STATISTICS")
        print("="*60)
        for s, d in self.stats.items():
            if d['count'] > 0:
                avg = d['total_time'] / d['count']
                print(f"\n{s.upper()} Strategy:")
                print(f"  Queries processed: {d['count']}")
                print(f"  Average time: {avg:.4f}s")
        print("\n" + "="*60)

In [ ]:
def run_tutorial():
    print("""
    META-REASONING AGENT TUTORIAL
    "When Should I Think Hard vs Answer Fast?"

    This agent demonstrates:
    1. Fast vs deep vs tool-based reasoning
    2. Choosing cognitive strategy
    3. Adaptive intelligence
    """)

    agent = MetaReasoningAgent()
    test_queries = [
        "What is the capital of France?",
        "Calculate 156 * 23",
        "Why do birds migrate south for winter?",
        "What is the latest news today?",
        "Hello!",
        "If all humans need oxygen and John is human, what can we conclude?",
    ]

    for q in test_queries:
        agent.process_query(q, verbose=True)
        time.sleep(0.5)

    agent.show_stats()
    print("\nTutorial complete!")
    print("• Meta-reasoning chooses how to think")
    print("• Different queries need different strategies")
    print("• Smart agents adapt reasoning dynamically\n")

In [1]:
if __name__ == "__main__":
    run_tutorial()


    ╔══════════════════════════════════════════════════════════════╗
    ║        META-REASONING AGENT TUTORIAL                         ║
    ║   "When Should I Think Hard vs Answer Fast?"                 ║
    ╚══════════════════════════════════════════════════════════════╝
    
    This agent demonstrates three key insights:
    
    1. NOT ALL QUERIES ARE EQUAL
       → Simple questions need fast answers
       → Complex problems need deep reasoning
       → Calculations need precise tools
    
    2. META-REASONING IS POWERFUL
       → Think about HOW to think BEFORE thinking
       → Choose the right cognitive strategy
       → Save resources on easy tasks
    
    3. ADAPTIVE INTELLIGENCE
       → Match strategy to problem difficulty
       → Use tools when precision matters
       → Balance speed and accuracy
    
    Let's see it in action!
    

QUERY: What is the capital of France?

🧠 META-REASONING:
   Complexity: simple
   Strategy: FAST
   Confidence: 95.00%
   Reasoning: